In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException

# caminho dos arquivos de modo global
caminho = os.getcwd()

options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", {
  "download.default_directory": fr"{caminho}\notas_ficais",
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True
})

navegador = webdriver.Chrome(options=options)

arquivo = caminho + r"\login.html"
navegador.get(arquivo)

# inserindo login se senha na primeira página
def realizar_login():
    dicionario_xpath_login = {
        "/html/body/div/form/input[1]": "123", 
        "/html/body/div/form/input[2]": "Teste", 
    }

    botao_loga_xpath =  "/html/body/div/form/button"

    # Digitando informações necessárias e clicando no botão
    for item in dicionario_xpath_login:
        navegador.find_element(By.XPATH, item).send_keys(dicionario_xpath_login[item])
    navegador.find_element(By.XPATH, botao_loga_xpath).click()

    time.sleep(1)

def planilha_e_listas():
    panilha_emissao = caminho + r"\NotasEmitir.xlsx"
    try:
        df = pd.read_excel(panilha_emissao)
        df = df.applymap(str)
    except Exception as e:
        print("Erro ao carregar o arquivo Excel:", e)

    lista_botoes = navegador.find_elements(By.TAG_NAME, "input")
    lista_colunas = df.columns

    return (lista_botoes, lista_colunas, df)

def realizar_preenchimento_campos():
    lista_botoes, lista_colunas, df = planilha_e_listas()
    df.reset_index(drop=True, inplace=True)

    for linha in df.index:
        coluna_idx = 0
        for coluna in lista_colunas:
            try:
                botao_nome = lista_botoes[coluna_idx].get_attribute("name")
                if "uf" not in botao_nome and coluna != "UF":
                    lista_botoes[coluna_idx].clear()
                    time.sleep(1)
                    navegador.find_element(By.NAME, botao_nome).send_keys(df.loc[linha, coluna])
                    time.sleep(1)
                elif coluna == "UF":
                    campo_uf = navegador.find_element(By.TAG_NAME, "select")
                    campo_uf = Select(campo_uf)
                    campo_uf.select_by_visible_text(df.loc[linha, coluna])
                    time.sleep(1)
                    coluna_idx -= 1
                coluna_idx += 1
            except NoSuchElementException as e:
                print(f"Elemento não encontrado: {botao_nome}")
            except Exception as e:
                print('Erro', e)
        navegador.find_element(By.XPATH, '//*[@id="formulario"]/button').click()


realizar_login()
realizar_preenchimento_campos()
navegador.quit()